In [9]:
!pip install opencv-python-headless -U
!pip install opencv-python -U

Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


### Carga de Imágenes

In [2]:
import cv2
import os

# Función para cargar imágenes desde una carpeta
def cargar_imagenes(carpeta):
    imagenes = []
    archivos = sorted([f for f in os.listdir(carpeta) if f.endswith('.jpg')])
    for archivo in archivos:
        img = cv2.imread(os.path.join(carpeta, archivo))
        if img is not None:
            imagenes.append(img)
    return imagenes

# Cargar imágenes de las carpetas
imagenes_cuar = cargar_imagenes('imgs/cuar')
imagenes_librero = cargar_imagenes('imgs/librero')

### Detección de Puntos de Interés y Matching

In [3]:
# Ejemplo usando ORB
orb = cv2.ORB_create()

def encontrar_y_emparejar(img1, img2):
    # Detectar puntos clave y descriptores
    kp1, des1 = orb.detectAndCompute(img1, None)
    kp2, des2 = orb.detectAndCompute(img2, None)

    # Crear un objeto BFMatcher y hacer matching
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
    matches = bf.match(des1, des2)
    matches = sorted(matches, key=lambda x: x.distance)

    return kp1, kp2, matches

# Ejemplo de emparejamiento entre la primera y segunda imagen de cuar
kp1, kp2, matches = encontrar_y_emparejar(imagenes_cuar[0], imagenes_cuar[1])


### Construcción de Homografías y Warping

In [4]:
import numpy as np

def aplicar_homografia(kp1, kp2, matches):
    # Extraer las coordenadas de los puntos emparejados
    src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)

    # Calcular la homografía
    H, _ = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
    return H

# Aplicar warping a la imagen
H = aplicar_homografia(kp1, kp2, matches)
img_warp = cv2.warpPerspective(imagenes_cuar[0], H, (imagenes_cuar[0].shape[1], imagenes_cuar[0].shape[0]))


### Blending de Imágenes

In [5]:
def blend_images(img1, img2):
    # Asumiendo que img1 e img2 están alineadas, podrías hacer un blending simple
    return cv2.addWeighted(img1, 0.5, img2, 0.5, 0)

# Ejemplo de blending
panorama = blend_images(img_warp, imagenes_cuar[1])


### Guardar y Mostrar Resultados

In [10]:
cv2.imwrite('panorama.jpg', panorama)
cv2.imshow('Panorama', panorama)
cv2.waitKey(0)
cv2.destroyAllWindows()

error: OpenCV(4.9.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1272: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvShowImage'
